# Scraping HealthGrades for References

In [1]:
    f!pip3 install lxml
#import all tools for project
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json
import numpy as np
import datetime
import matplotlib.pyplot as plt
import re
from time import sleep

You should consider upgrading via the '/usr/local/bin/python3.8 -m pip install --upgrade pip' command.


In [2]:
#example for exactly one doctor

url = 'https://www.healthgrades.com/physician/dr-craig-ehrensing-yxw7q'
def get_reviews_old(url):
    '''
    Takes in a url to any specific doctor and returns all of their reviews as strs in a list
    '''
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    response = requests.get(url, headers=headers)

    #use Beautiful Soup to prettify the content
    soup = BeautifulSoup(response.content)
    soup.prettify()
    
    gender = None
    #determine gender 
    headdiv = soup.findAll("div", {"class":"summary-row summary-row-top"})
    if headdiv:
        for tag in headdiv:  
            genderInfo = tag.findAll("span", {"data-qa-target":"ProviderDisplayGender"})
            genderStr = str(genderInfo)
            result = re.search('<span data-qa-target="ProviderDisplayGender"> <!-- -->(.*)<!-- --> </span>', genderStr)
            gender = result.group(1)
    else:
        headdiv = soup.findAll("div", {"class": "summary-designated-header-image-left"})
        for tag in headdiv:  
            genderInfo = tag.findAll("span", {"data-qa-target":"ProviderDisplayGender"})
            genderStr = str(genderInfo)
            result = re.search('<span data-qa-target="ProviderDisplayGender"> <!-- -->(.*)<!-- --> </span>', genderStr)
            gender = result.group(1)
            
    #retrieve all comments
    commentdivs = soup.findAll("div", {"class": "c-single-comment__comment"})
    commentdivs
    reviews = []
    for div in commentdivs:
        s = str(div)
        s = s[1:]
        start = s.find(">")+1
        end = s.find("<")
        substring = s[start:end]
        reviews.append(substring)
    return gender, reviews

In [3]:
linkx = 'https://www.healthgrades.com/dentist/dr-lauren-rivet-3qyd9'

def get_reviews(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    response = requests.get(url, headers=headers)

    #use Beautiful Soup to prettify the content
    soup = BeautifulSoup(response.content)
    soup.prettify()

    gender = None

    #retrieve all comments
    commentdivs = soup.findAll("div", {"class": "c-single-comment__comment"})
    commentdivs
    reviews = []
    for div in commentdivs:
        s = str(div)
        s = s[1:]
        start = s.find(">")+1
        end = s.find("<")
        substring = s[start:end]
        reviews.append(substring)
    #Find gender 
    personal = soup.findAll("span", {"data-qa-target": "ProviderDisplayGender"})
    gender = re.search("(?<=>).*?(?=</)", str(personal[0]))
    return gender.group(0), reviews

In [4]:
get_reviews('https://www.healthgrades.com/dentist/dr-lauren-rivet-3qyd9')

('Female',
 ['During my visit, I was greeted with a very friendly and welcoming environment. I was very comfortable the whole time there. I was informed step by step everything that was going to take place, which gave me comfort. I was fully satisfied with the work performed.',
  'Very good service and her assistant was very helpful.',
  'Everyone was amazing. I was anxious about my cleaning, but everyone in the office was soothing &amp; supportive. Truly made a visit to the dentist a delightful experience.',
  'Wonderful!!!! Everyone was nice and very professional.',
  'Knowledge, friendly staff and dentist.',
  'Always gentle and understanding of our anxieties! Dr. Rivet is an excellent dentist and her staff was warm and welcoming.',
  'Dr. Rivet, has patience and she was very heartwarming. I was nervous and she assured me everything was going to be alright and it was wonderful.  Her staff is exceptionally polite and welcoming. This dental office has first class experience. If your l

In [5]:
url2 = 'https://www.healthgrades.com/physician/dr-paula-hall-ypy2q'
get_reviews(url2)

('Female',
 ["Dr. Hall is an excellent physician with a wonderful bedside manner! She made me feel very at ease as to what was going to happen during my procedure and afterward. She goes over the results so you understand what has taken place. \nThe staff is also great in regards to the post-op instructions.   Dr. Hall has done my routine colonoscopy's for over 10 years- I would highly recommend her to others! ",
  'Dr. Hall is an excellent physician with a wonderful bedside manner! She made me feel very at ease as to what was going to happen during my procedure and afterward. Her staff was also great in regard to the pre-op instructions. So glad that I chose Dr. Hall for my routine colonoscopy- I will recommend her highly to others! ',
  'Dr. Hall is an outstanding physician. She has excellent bedside manner, takes time to thoroughly explain the procedure, what to expect, describes what she will do if she encounters any abnormalities and answers any questions you have.  I highly recom

In [19]:
#  TESTS FOR one PAGE

linkA = 'https://www.healthgrades.com/find-a-doctor/a-z/g-1'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
response = requests.get(linkA, headers=headers)
soup = BeautifulSoup(response.content)
soup.prettify()


lnks = soup.findAll("a")
docsA = []
for i in range(len(lnks)):
    trial = re.findall(r"<a href=\"/physician+/+", str(lnks[i]))
    if trial == ['<a href="/physician/']:

        cut = re.findall(r'</?a(|\s+[^>]+)>',str(lnks[i]))

        docsA.append(cut[0][7:-1])
docsA


['/physician/dr-manuel-g-yy4kw',
 '/physician/dr-maaza-g-amlak-ggdqx',
 '/physician/dr-michele-gains-xql7x',
 '/physician/dr-kweku-grant-acquah-354l2',
 '/physician/dr-sanjeev-grewal-2564w',
 '/physician/dr-thomas-gaab-3c4tp',
 '/physician/dr-imran-gaad-6f3a8wb874',
 '/physician/dr-hussein-gaafary-y9vgmdz',
 '/physician/dr-hany-gaafer-y5ykn',
 '/physician/dr-dorothy-gaal-yjxlr',
 '/physician/dr-imre-gaal-3643m',
 '/physician/dr-irina-gaal-2hjx5',
 '/physician/dr-james-gaal-2wsgm',
 '/physician/dr-jordan-gaal-xymqq4y',
 '/physician/dr-karl-gaal-ylms5',
 '/physician/dr-peter-gaal-ydy95',
 '/physician/dr-stephen-gaal-yhvlc',
 '/physician/dr-wade-gaal-gf7r9',
 '/physician/dr-ajay-gaalla-2lt6p',
 '/physician/dr-ryan-gaalswyk-x5vq6',
 '/physician/dr-jalong-gaan-yf7vv',
 '/physician/dr-carlo-gaanan-yc8m3',
 '/physician/dr-david-gaar-ysgdf',
 '/physician/dr-edwin-gaar-xc9b7',
 '/physician/dr-gregory-gaar-ypjyx',
 '/physician/dr-joseph-gaar-2m5lh',
 '/physician/dr-laini-gaar-gf6gj',
 '/physicia

In [29]:
# stores links to a-z pages
AtoZlinks = []

links.append('https://www.healthgrades.com/find-a-doctor/a-z/a-1')

AtoZ = soup.findAll("div", {"class":"alpha-container__body"})
for page in AtoZ[0].findAll("a", href = True):
    suffix = page["href"]
    AtoZlinks.append('https://www.healthgrades.com/find-a-doctor/a-z/' + suffix)
AtoZlinks

['https://www.healthgrades.com/find-a-doctor/a-z/a-1',
 'https://www.healthgrades.com/find-a-doctor/a-z/b-1',
 'https://www.healthgrades.com/find-a-doctor/a-z/c-1',
 'https://www.healthgrades.com/find-a-doctor/a-z/d-1',
 'https://www.healthgrades.com/find-a-doctor/a-z/e-1',
 'https://www.healthgrades.com/find-a-doctor/a-z/f-1',
 'https://www.healthgrades.com/find-a-doctor/a-z/h-1',
 'https://www.healthgrades.com/find-a-doctor/a-z/i-1',
 'https://www.healthgrades.com/find-a-doctor/a-z/j-1',
 'https://www.healthgrades.com/find-a-doctor/a-z/k-1',
 'https://www.healthgrades.com/find-a-doctor/a-z/l-1',
 'https://www.healthgrades.com/find-a-doctor/a-z/m-1',
 'https://www.healthgrades.com/find-a-doctor/a-z/n-1',
 'https://www.healthgrades.com/find-a-doctor/a-z/o-1',
 'https://www.healthgrades.com/find-a-doctor/a-z/p-1',
 'https://www.healthgrades.com/find-a-doctor/a-z/q-1',
 'https://www.healthgrades.com/find-a-doctor/a-z/r-1',
 'https://www.healthgrades.com/find-a-doctor/a-z/s-1',
 'https://

In [30]:
doctors = []
for page in AtoZlinks:
    
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    response = requests.get(page, headers=headers)
    soup = BeautifulSoup(response.content)
    soup.prettify()

    lnks = soup.findAll("a")
    for i in range(len(lnks)):
        trial = re.findall(r"<a href=\"/physician+/+", str(lnks[i]))
        if trial == ['<a href="/physician/']:
            cut = re.findall(r'</?a(|\s+[^>]+)>',str(lnks[i]))
            doctors.append("https://www.healthgrades.com" + cut[0][7:-1])
doctors

['https://www.healthgrades.com/physician/dr-justin-a-de-boer-cjybz',
 'https://www.healthgrades.com/physician/dr-ruben-a-6jx16p1158',
 'https://www.healthgrades.com/physician/dr-enenge-abodjedi-25pd2',
 'https://www.healthgrades.com/physician/dr-yousif-a-rahim-35982',
 'https://www.healthgrades.com/physician/dr-melanie-aaberg-3vtmd',
 'https://www.healthgrades.com/physician/dr-randal-aaberg-y5snb',
 'https://www.healthgrades.com/physician/dr-thomas-aaberg-ynfm8',
 'https://www.healthgrades.com/physician/dr-thomas-aaberg-xxkf9',
 'https://www.healthgrades.com/physician/dr-afeera-aabida-xr46l',
 'https://www.healthgrades.com/physician/dr-meghan-aabo-xylnfmj',
 'https://www.healthgrades.com/physician/dr-stella-aaboe-xj54y',
 'https://www.healthgrades.com/physician/dr-aazy-aaby-33pd2',
 'https://www.healthgrades.com/physician/dr-douglas-aach-2r68l',
 'https://www.healthgrades.com/physician/dr-james-aachen-winans-bu1ylnq531',
 'https://www.healthgrades.com/physician/dr-venkat-aachi-yhmh5',


In [31]:
# total data points:

print("Number of doctors collected: ", len(doctors))

Number of doctors collected:  5136


In [32]:
#scrape reviews from each doctor page

revs = {}
numRev = 1
for url in doctors:
    try:
        (gender, commentList) = get_reviews(url)
        for comment in commentList:
            revs[comment] = gender
    except:
        print('not working: ', url)
        
    #tracks progress of scraping, takes approx 45 mins
    print('.', numRev, end = "\r", flush = True)
    numRev += 1

revs

. 1 . 2 . 3 . 4 . 5 . 6 . 7 . 8 . 9 . 10 . 11 . 12 . 13 . 14 . 15 . 16 . 17 . 18 . 19 . 20 . 21 . 22 . 23 . 24 . 25 . 26 . 27 . 28 . 29 . 30 . 31 . 32 . 33 . 34 . 35 . 36 . 37 . 38 . 39 . 40 . 41 . 42 . 43 . 44 . 45 . 46 . 47 . 48 . 49 . 50 . 51 . 52 . 53 . 54 . 55 . 56 . 57 . 58 . 59 . 60 . 61 . 62 . 63 . 64 . 65 . 66 . 67 . 68 . 69 . 70 . 71 . 72 . 73 . 74 . 75 . 76 . 77 . 78 . 79 . 80 . 81 . 82 . 83 . 84 . 85 . 86 . 87 . 88 . 89 . 90 . 91 . 92 . 93 . 94 . 95 . 96 . 97 . 98 . 99 . 100 . 101 . 102 . 103 . 104 . 105 . 106 . 107 . 108 . 109 . 110 . 111 . 112 . 113 . 114 . 115 . 116 . 117 . 118 . 119 . 120 . 121 . 122 . 123 . 124 . 125 . 126 . 127 . 128 . 129 . 130 . 131 . 132 . 133 . 134 . 135 . 136 . 137 . 138 . 139 . 140 . 141 . 142 . 143 . 144 . 145 . 146 . 147 . 148 . 149 . 150 . 151 . 152 . 153 . 154 . 155 . 156 . 157 . 158 . 159 . 160 . 161 . 162 . 163 . 164 . 165 . 166 . 167 . 168 . 169 . 170 . 171 . 172 . 173 . 174 . 175 . 176 . 177 . 178 . 179 . 180 . 181 . 182 . 183 . 184 . 18

. 1330 . 1331 . 1332 . 1333 . 1334 . 1335 . 1336 . 1337 . 1338 . 1339 . 1340 . 1341 . 1342 . 1343 . 1344 . 1345 . 1346 . 1347 . 1348 . 1349 . 1350 . 1351 . 1352 . 1353 . 1354 . 1355 . 1356 . 1357 . 1358 . 1359 . 1360 . 1361 . 1362 . 1363 . 1364 . 1365 . 1366 . 1367 . 1368 . 1369 . 1370 . 1371 . 1372 . 1373 . 1374 . 1375 . 1376 . 1377 . 1378 . 1379 . 1380 . 1381 . 1382 . 1383 . 1384 . 1385 . 1386 . 1387 . 1388 . 1389 . 1390 . 1391 . 1392 . 1393 . 1394 . 1395 . 1396 . 1397 . 1398 . 1399 . 1400 . 1401 . 1402 . 1403 . 1404 . 1405 . 1406 . 1407 . 1408 . 1409 . 1410 . 1411 . 1412 . 1413 . 1414 . 1415 . 1416 . 1417 . 1418 . 1419 . 1420 . 1421 . 1422 . 1423 . 1424 . 1425 . 1426 . 1427 . 1428 . 1429 . 1430 . 1431 . 1432 . 1433 . 1434 . 1435 . 1436 . 1437 . 1438 . 1439 . 1440 . 1441 . 1442 . 1443 . 1444 . 1445 . 1446 . 1447 . 1448 . 1449 . 1450 . 1451 . 1452 . 1453 . 1454 . 1455 . 1456 . 1457 . 1458 . 1459 . 1460 . 1461 . 1462 . 1463 . 1464 . 1465 . 1466 . 1467 . 1468 . 1469 . 1470 . 1471 . 1472

. 2501 . 2502 . 2503 . 2504 . 2505 . 2506 . 2507 . 2508 . 2509 . 2510 . 2511 . 2512 . 2513 . 2514 . 2515 . 2516 . 2517 . 2518 . 2519 . 2520 . 2521 . 2522 . 2523 . 2524 . 2525 . 2526 . 2527 . 2528 . 2529 . 2530 . 2531 . 2532 . 2533 . 2534 . 2535 . 2536 . 2537 . 2538 . 2539 . 2540 . 2541 . 2542 . 2543 . 2544 . 2545 . 2546 . 2547 . 2548 . 2549 . 2550 . 2551 . 2552 . 2553 . 2554 . 2555 . 2556 . 2557 . 2558 . 2559 . 2560 . 2561 . 2562 . 2563 . 2564 . 2565 . 2566 . 2567 . 2568 . 2569 . 2570 . 2571 . 2572 . 2573 . 2574 . 2575 . 2576 . 2577 . 2578 . 2579 . 2580 . 2581 . 2582 . 2583 . 2584 . 2585 . 2586 . 2587 . 2588 . 2589 . 2590 . 2591 . 2592 . 2593 . 2594 . 2595 . 2596 . 2597 . 2598 . 2599 . 2600 . 2601 . 2602 . 2603 . 2604 . 2605 . 2606 . 2607 . 2608 . 2609 . 2610 . 2611 . 2612 . 2613 . 2614 . 2615 . 2616 . 2617 . 2618 . 2619 . 2620 . 2621 . 2622 . 2623 . 2624 . 2625 . 2626 . 2627 . 2628 . 2629 . 2630 . 2631 . 2632 . 2633 . 2634 . 2635 . 2636 . 2637 . 2638 . 2639 . 2640 . 2641 . 2642 . 2643

. 3672 . 3673 . 3674 . 3675 . 3676 . 3677 . 3678 . 3679 . 3680 . 3681 . 3682 . 3683 . 3684 . 3685 . 3686 . 3687 . 3688 . 3689 . 3690 . 3691 . 3692 . 3693 . 3694 . 3695 . 3696 . 3697 . 3698 . 3699 . 3700 . 3701 . 3702 . 3703 . 3704 . 3705 . 3706 . 3707 . 3708 . 3709 . 3710 . 3711 . 3712 . 3713 . 3714 . 3715 . 3716 . 3717 . 3718 . 3719 . 3720 . 3721 . 3722 . 3723 . 3724 . 3725 . 3726 . 3727 . 3728 . 3729 . 3730 . 3731 . 3732 . 3733 . 3734 . 3735 . 3736 . 3737 . 3738 . 3739 . 3740 . 3741 . 3742 . 3743 . 3744 . 3745 . 3746 . 3747 . 3748 . 3749 . 3750 . 3751 . 3752 . 3753 . 3754 . 3755 . 3756 . 3757 . 3758 . 3759 . 3760 . 3761 . 3762 . 3763 . 3764 . 3765 . 3766 . 3767 . 3768 . 3769 . 3770 . 3771 . 3772 . 3773 . 3774 . 3775 . 3776 . 3777 . 3778 . 3779 . 3780 . 3781 . 3782 . 3783 . 3784 . 3785 . 3786 . 3787 . 3788 . 3789 . 3790 . 3791 . 3792 . 3793 . 3794 . 3795 . 3796 . 3797 . 3798 . 3799 . 3800 . 3801 . 3802 . 3803 . 3804 . 3805 . 3806 . 3807 . 3808 . 3809 . 3810 . 3811 . 3812 . 3813 . 3814

. 4843 . 4844 . 4845 . 4846 . 4847 not working:  https://www.healthgrades.com/physician/dr-mais-yacoub-y9qlslz
. 4848 . 4849 . 4850 . 4851 . 4852 . 4853 . 4854 . 4855 . 4856 . 4857 . 4858 . 4859 . 4860 . 4861 . 4862 . 4863 . 4864 . 4865 . 4866 . 4867 . 4868 . 4869 . 4870 . 4871 . 4872 . 4873 . 4874 . 4875 . 4876 . 4877 . 4878 . 4879 . 4880 . 4881 . 4882 . 4883 . 4884 . 4885 . 4886 . 4887 . 4888 . 4889 . 4890 . 4891 . 4892 . 4893 . 4894 . 4895 . 4896 . 4897 . 4898 . 4899 . 4900 . 4901 . 4902 . 4903 . 4904 . 4905 . 4906 . 4907 . 4908 . 4909 . 4910 . 4911 . 4912 . 4913 . 4914 . 4915 . 4916 . 4917 . 4918 . 4919 . 4920 . 4921 . 4922 . 4923 . 4924 . 4925 . 4926 . 4927 . 4928 . 4929 . 4930 . 4931 . 4932 . 4933 . 4934 . 4935 . 4936 . 4937 . 4938 . 4939 . 4940 . 4941 . 4942 . 4943 . 4944 . 4945 . 4946 . 4947 . 4948 . 4949 . 4950 . 4951 . 4952 . 4953 . 4954 . 4955 . 4956 . 4957 . 4958 . 4959 . 4960 . 4961 . 4962 . 4963 . 4964 . 4965 . 4966 . 4967 . 4968 . 4969 . 4970 . 4971 . 4972 . 4973 . 4974 

{'Very good': 'Male',
 'Exelent he really care about you, he is very professional,  and , compassionate.  Thanks Dr A ‘ dbojeri.': 'Male',
 'Dr. A-Rahim was very knowledgeable about my long term condition and spent a great deal of time explaining my treatment plan to me and my husband.': 'Male',
 'Horrible physician treats you like a kid. Pts c/o in er is never addressed by this md , she dances around anything you come in for. You will be told call your internal medicine dr because nothing is wrong, horrible bedside manners and argues with basic pt care ': 'Female',
 'Dr Aaberg has been a very good dr for my husband  Very kind and answers all your questions ': 'Male',
 'Dr. Aaberg has been treating my macular degeneration for five years. I could not ask for a better, smarter, professional doctor. I highly recommend him and his outstanding staff.  They all care about me as a person and as a patient.': 'Male',
 'I am privileged to have Dr Aaberg in charge of my eye care.': 'Male',
 'I ha

In [33]:
# number of data points:
print("Number of reviews collected: ", len(revs))

Number of reviews collected:  8872


In [34]:
reviewsByGender = pd.DataFrame(list(revs.items()),columns = ['Review','Gender'])
reviewsByGender

,Review,Gender
0,Very good,Male
1,"Exelent he really care about you, he is very p...",Male
2,Dr. A-Rahim was very knowledgeable about my lo...,Male
3,Horrible physician treats you like a kid. Pts ...,Female
4,Dr Aaberg has been a very good dr for my husba...,Male
...,...,...
8867,Very helpful and sincere about my pain needs! ...,Male
8868,"Several of my children see Dr. Zach, she is a ...",Female
8869,Amazing Pediatric Neurologist stays up to date...,Female
8870,Dr. Zach has been a Godsend for our daughter! ...,Female


# Determining the Frequency of Gendered Language

### References
Scikit-learn: Machine Learning in Python, Pedregosa et al., JMLR 12, pp. 2825-2830, 2011.

Dr. Mattei, Tulane University. SimpleText Notebook. 2020. 
Bo Pang and Lillian Lee, A Sentimental Education: Sentiment Analysis Using Subjectivity Summarization Based on Minimum Cuts, Proceedings of ACL 2004.

In [61]:
# Vectorize the whole thing...
import sklearn
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


# Vectorize and play with token sizes...
vec = TfidfVectorizer(min_df = 2, 
                      max_df = 0.98, 
                      ngram_range=(1,1)) # play with min_df and max_df

reviews = reviewsByGender['Review']

# transform this into a sparse vector!
vec.fit(reviews)
tf_idf_sparse = vec.transform(reviews)
tf_idf_sparse

#only 222 unique words due to min_df 0.03 --> change to 2, maybe change to 5 (could get meaningless words)

<8872x222 sparse matrix of type '<class 'numpy.float64'>'
	with 204298 stored elements in Compressed Sparse Row format>

In [62]:
#get list of cleaned, stemmed adjectives, for later analysis. 
vect = CountVectorizer()
vect.fit(revs)
words = vect.get_feature_names()

adjectives = []
stop_words = list(set(stopwords.words('english')))

# remove stopwords 
stopped = [w for w in words if not w in stop_words]
    
# parts of speech tagging for each word 
pos = nltk.pos_tag(stopped)

# make a list of  all adjectives identified by the allowed word types list above
for w in pos:
    if w[1][0] == 'J':
        adjectives.append(w[0].lower())

In [63]:
print(tf_idf_sparse[0, :])

  (0, 199)	0.5117404335375177
  (0, 79)	0.8591401100419147


In [64]:
# We can now use this to classify the reviews!! but we need to test/train split again.

# Split..
X_train, X_test, y_train, y_test = train_test_split(tf_idf_sparse, 
                                                    reviewsByGender['Gender'], 
                                                    test_size=0.2)

In [65]:
logisticRegr = LogisticRegression(max_iter=100000, class_weight='balanced') 
model = logisticRegr.fit(X_train, y_train)

/Users/laurensussman/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [66]:
logisticRegr.classes_

array(['Female', 'Male'], dtype=object)

In [67]:
logisticRegr.coef_[:,0]

array([0.26186066])

In [68]:
# Make a dataframe with the words, coefficients, and classes...
recs = []

for w,i in vec.vocabulary_.items():
    recs.append([str(w)] + list(logisticRegr.coef_[:,i]))
# If we only have one class then we only get weight..
# df_weights = pd.DataFrame(tripples, columns=['word']+list(logisticRegr.classes_))
df_weights = pd.DataFrame(recs, columns=['word', 'weight'])

In [69]:
df_weights.sort_values('weight', ascending=False)[:25]

,word,weight
2,he,12.841362
51,his,8.664204
50,him,7.207713
154,surgery,2.808157
52,staff,1.422823
175,pain,1.242797
196,helpful,1.123723
35,because,0.984665
211,done,0.957400
12,was,0.949824


In [70]:
#print only adjectives
#male adjectives
maleAdjectives = df_weights[df_weights['word'].isin(adjectives)].sort_values('weight', ascending=False)[:25]
maleAdjectives

,word,weight
16,great,0.692036
64,much,0.508382
1,good,0.487285
204,enough,0.481534
89,sure,0.435180
141,medical,0.388757
8,professional,0.313120
66,explain,0.274041
219,able,0.261861
163,friendly,0.184121


In [71]:
df_weights.sort_values('weight', ascending=True)[:25]

,word,weight
28,she,-10.636285
173,her,-8.497605
13,knowledgeable,-0.995259
118,patients,-0.992587
177,love,-0.992075
159,felt,-0.974966
11,dr,-0.948742
185,concerns,-0.929725
45,not,-0.911766
155,make,-0.884010


In [72]:
#female adjectives
femaleAdjectives = df_weights[df_weights['word'].isin(adjectives)].sort_values('weight', ascending=True)[:25]
#maybe take out adjective specification
#vocabulary being pruned down too aggressively
femaleAdjectives

,word,weight
13,knowledgeable,-0.995259
188,wonderful,-0.873516
76,best,-0.543493
194,amazing,-0.532937
22,physician,-0.507958
208,nice,-0.432374
215,comfortable,-0.367210
214,new,-0.233826
171,several,-0.088495
168,easy,-0.001302


In [73]:
#verify results
agentic_terms = ['assertive','confident','aggressive','ambitious','dominan','forceful','independent','daring','outspoken','intellectua','earn','gain','know','insight','think']
communal_terms = ['sympathetic','kind','help','affection','sensitive','nurtur','agreeab','tactful','interpersonal','warm','car','tactful']
socio_communal = ['husband','wife','kid','babies','brother','child','colleague','family']

print('Results for most commonly found female adjectives:')
print('Agentic: ', femaleAdjectives['word'].isin(agentic_terms).sum())
print('Communal: ', femaleAdjectives['word'].isin(communal_terms).sum())
print('Socio-communal: ', femaleAdjectives['word'].isin(socio_communal).sum())
print('')
print('Results for most commonly found male adjectives:')
print('Agentic: ', maleAdjectives['word'].isin(agentic_terms).sum())
print('Communal: ', maleAdjectives['word'].isin(communal_terms).sum())
print('Socio-communal: ', maleAdjectives['word'].isin(socio_communal).sum())

Results for most commonly found female adjectives:
Agentic:  0
Communal:  0
Socio-communal:  0

Results for most commonly found male adjectives:
Agentic:  0
Communal:  0
Socio-communal:  0


In [ ]:
#can use medical reviews to run with recommnedation letter reviews to predict which is which, then throw out 
#heavily medical terms

# recommendation system
# maybe seek out positive words that don't distinguish between male and female
# 4-class problem rather than two-class problem: take in star data and add pos/neg dimension to the data
# word vector stuff from NLP class: use language models and figure out how often it appears in male context vs
# female context. Ex: look at similarities between words: compare "leadership" to "man/men" 

#doubt raisers may not come out from this analysis: "she may not be the best in her class..."
# subtle, hard to catch -> maybe the word "not"
# append "not" to nearest adjective, negated adjectives might indicate doubt raisers
# hedging terms to weaken a statement (weasel words): refusing to take a stand/hesitant
# "somewhat", "mostly"

# how to rephrase it: female-correlated to neutral, remove hedging terms
# take an education based approach: "this can be conveyed as ___. If that is not your intention, consider ___"
# crowdsourcing: emphasize anonymization